In [2]:
from glob import glob
import os

labels= glob('/media/mws/DATA/fire_videos/Training/*/*.json')
images= glob('/media/mws/DATA/fire_videos/Training/*/*.jpg')
# images2 

In [3]:
print(len(labels))
print(len(images))

1369963
1284510


In [4]:
a = set()
for i in labels:
    a.add(os.path.basename(i)[:-4] + 'jpg')

In [5]:
b = set()
for i in images:
    b.add(os.path.basename(i))

In [6]:
len(a)

1369963

In [7]:
len(b)

1284510

In [8]:
c = list(a-b)

In [9]:
b-a

{'S3-N0806MS00922.jpg',
 'S3-N0806MS00923.jpg',
 'S3-N0806MS00924.jpg',
 'S3-N0806MS00925.jpg',
 'S3-N0806MS00926.jpg',
 'S3-N0806MS00927.jpg',
 'S3-N0806MS00928.jpg',
 'S3-N0806MS00929.jpg',
 'S3-N0806MS00930.jpg',
 'S3-N0806MS00931.jpg',
 'S3-N0806MS00932.jpg',
 'S3-N0806MS00933.jpg',
 'S3-N0806MS00934.jpg',
 'S3-N0806MS00935.jpg',
 'S3-N0806MS00936.jpg',
 'S3-N0806MS00937.jpg',
 'S3-N0806MS00938.jpg',
 'S3-N0806MS00939.jpg',
 'S3-N0806MS00940.jpg',
 'S3-N0806MS00941.jpg',
 'S3-N0806MS00942.jpg',
 'S3-N0806MS00943.jpg',
 'S3-N0806MS00944.jpg',
 'S3-N0806MS00945.jpg',
 'S3-N0806MS00946.jpg',
 'S3-N0806MS00947.jpg',
 'S3-N0806MS00948.jpg',
 'S3-N0806MS00949.jpg',
 'S3-N0806MS00950.jpg',
 'S3-N0806MS00951.jpg',
 'S3-N0806MS00952.jpg',
 'S3-N0806MS00953.jpg',
 'S3-N0806MS00954.jpg',
 'S3-N0806MS00955.jpg',
 'S3-N0806MS00956.jpg',
 'S3-N0806MS00957.jpg',
 'S3-N0806MS00958.jpg',
 'S3-N0806MS00959.jpg',
 'S3-N0806MS00960.jpg',
 'S3-N0806MS00961.jpg',
 'S3-N0806MS00962.jpg',
 'S3-N0806MS0096

In [10]:
len(c)

85750

In [11]:
len(b)

1284510

In [12]:
print(len(set(labels)))
print(len(set(images)))

1369963
1284510


In [13]:
from multiprocessing import Process

In [14]:
import json 
import os 
from PIL import Image
import shutil 
from tqdm import tqdm 

In [15]:
os.path.dirname(i).split('씬')[0] + '씬'

'/media/mws/DATA/fire_videos/Training/화재씬'

In [16]:
# for i in tqdm(images):
#     des = os.path.dirname(i).split('씬')[0] + '씬'
#     try:
#         shutil.move(i,des)
#     except:
#         continue



In [17]:
def cvt_yoloformat(xyxy,w,h):
    box_w = xyxy[2] - xyxy[0]
    box_h = xyxy[3] - xyxy[1]
    return [(xyxy[0] + box_w/2)/w ,(xyxy[1] + box_h/2)/h, box_w/w, box_h/h ]

In [22]:
count1 = 0
count2 = 0
strange_img = []
flag = False
for i in tqdm(labels):
    if flag:
        break
    try:
        with open(i, 'r') as f:
            datas = json.load(f)
    except:
        count1 +=1
        continue
    # print(i)

    # img_name = datas['image']['filename']
    img_name = os.path.basename(i)[:-4] +'jpg'
    img_path = i.replace('[라벨]', '')[:-4] + 'jpg'

    if not os.path.exists(img_path):
        continue

    try:
        img = Image.open(img_path)
    except:
        strange_img.append(img_path)
        continue

    width,height = img.size
    # img_path = i.replace('라벨','원천').split('')[0] + '/' + img_name

    annos = datas['annotations']

    new_datas = []
    # try:
    for j in range(len(annos)):

        if list(annos[j].keys())[3] == 'polygon':
            poly = annos[j]['polygon']
            max_x, max_y, min_x, min_y = 0,0,0,0
            
            for k in poly:
                if len(k) <=1:
                    continue
                p0 , p1 = int(k[0]) , int(k[1])
                if(p1 >= 1081):
                    print(k)
                    flag = True
                    break
                max_x = max(max_x,p0)
                max_y = max(max_y,p1)
                min_x = min(min_x,p0)
                min_y = min(min_y,p1)

            bbox = [min_x,min_y,max_x,max_y]
            xywh = cvt_yoloformat(bbox,width,height)
            if (xywh[2] > 1) | (xywh[3] > 1):
                print(bbox)
                print(xywh)
                print(i)
                print(img.size)
                print(poly)
                
                flag = True
                break
            
            # print(xywh)
            # new_datas.append(bbox)

            abc = [int(annos[j]['class']), *xywh]
            new_datas.append(abc)

        else:

            bbox = annos[j]['box']
            # print(bbox)
            xywh = cvt_yoloformat(bbox,width,height)
            abc = [int(annos[j]['class']), *xywh]
            new_datas.append(abc)
    # except:
    #     count2 +=1
    lab_dir = os.path.dirname(i).replace('[', '').replace(']','_')
    # print(lab_dir + os.path.basename(i)[:-4] + 'txt')

    with open(lab_dir +'/' + os.path.basename(i)[:-4] + 'txt','w') as f:
        for j in new_datas:
            j = [str(k) for k in j]
            f.write(' '.join(j) + '\n')

print(count1)
print(count2)


100%|██████████| 1369963/1369963 [12:54<00:00, 1768.37it/s]

0
0


In [23]:
print(count1)
print(count2)

0
0


In [139]:
a = ['1','2','3','4']
str(a)


"['1', '2', '3', '4']"

In [24]:
txts = glob('/media/mws/DATA/fire_videos/Training/*/*.txt')
imgs1 = glob('/media/mws/DATA/fire_videos/Training/화재씬/*.jpg')
imgs2 = glob('/media/mws/DATA/fire_videos/Training/유사씬/*.jpg')
imgs3 = glob('/media/mws/DATA/fire_videos/Training/무관씬/*.jpg')


In [27]:
a= ['1' ,'2','3']
' '.join(a[1:])

'2 3'

In [25]:
print(len(imgs1))
print(len(imgs2))
print(len(imgs3))
len(txts)

493159
548000
243351


1284212

In [ ]:
print(len(imgs1))
print(len(imgs2))
print(len(imgs3))
len(txts)

493159
548000
243351


In [19]:
len(txts)

1284212

In [32]:
len(txts)

1267820

In [38]:
len(txts)

1284212

In [21]:
imgses  = imgs1 + imgs2 + imgs3

In [22]:
len(imgses)

1284510

In [ ]:
imgs2 = glob('')

In [21]:
len(txts)

476766